# Pratique, le Dataset du Titanic

**Le but est prévoire si une personne survit ou non**

C'est donc un problème de **Classification**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')

In [4]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


On observe que nous avons 3 types de variables parmis nos colonnes, 
- **Continues** : "*age*" et "*fare*"
- **Discrètes** : "*pclass*", "*survived*", ...
- **Des Charactères** : "*sex*", "*who*", "*class*", ...

Pour gérer ces différents types de variables entre nos colonnes nous allons utliser `make_column_transformer`pour traiter chaque colonnes comme il se doit.


## Preparer le Dataset

- Sépare la colonne des prédictions

In [5]:
y = titanic['survived']
X = titanic.drop('survived', axis=1)

In [6]:
from sklearn.model_selection import train_test_split

In [38]:
x_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [39]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((712, 14), (712,), (712,), (179,))

## Pre-processing

In [40]:
titanic.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


### Pipeline avancé

Selection et séparation de nos colonnes en focntion de leur variables **Catégoriels** ou **Numériques**

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [44]:
numeric_var = ['pclass', 'age', 'fare']
categorie_var = ['sex', 'deck', 'alone']

#### Pipeline variables Continue
- Normalisation avec `StandardScaler()` 
- Supprimer les valeurs manquantes avec `SimpleImputer()`

In [43]:
numericale_pipeline = make_pipeline(SimpleImputer(strategy='mean'),
                                    StandardScaler())

#### Pipeline variables Catégoriel 
- Gestion des valeurs manquantes avec `SimpleImputer()`et leur remplacement par les valles plus fréquentes
- Encodage avec `OneHotEncoder()`

In [42]:
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                     OneHotEncoder())

#### **`Make_column_transformer()`**
Nous allons passer nos deux traitement de pipeline pour nos différentes variables dans `make_column_transformer` pour les compiler à la suite

In [45]:
from sklearn.compose import make_column_transformer

In [46]:
preprocessor = make_column_transformer((numericale_pipeline, numeric_var), 
                       (categorical_pipeline, categorie_var))

### Pipeline Final

On vient à sont tour mettre notre Objet *processor* dans une nouvelle *Pipeline* pour y ajouté le modèle de ML de notre choix

In [47]:
from sklearn.linear_model import SGDClassifier

In [48]:
model = make_pipeline(preprocessor, SGDClassifier())

In [49]:
model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                             

In [50]:
model.score(X_test, y_test)

0.7821229050279329